In [1]:
# 读取图像信息
import shutil
import os


training_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/training/'
train_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/train_data/'
dirs = os.listdir(training_path)
for item in dirs:
    item_label = item.split('_')[0]
    des_path = train_data_path+item_label
    if not os.path.exists(des_path):
        os.makedirs(des_path)
    shutil.copyfile(os.path.join(training_path, item), os.path.join(des_path, item))


testing_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/testing/'
test_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/test_data/'
dirs = os.listdir(testing_path)
for item in dirs:
    item_label = item.split('_')[0]
    des_path = test_data_path+item_label
    if not os.path.exists(des_path):
        os.makedirs(des_path)
    shutil.copyfile(os.path.join(testing_path, item), os.path.join(des_path, item))

In [2]:
# 
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
# 解决运行时服务挂掉的问题
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [3]:
# classes = 10
# batch_size = 64
# epochs = 20

classes = 10
batch_size = 2
epochs = 2

def resnet_block(inputs, 
                 num_filters=16, 
                 kernel_size=3,
                 strides=1, 
                 activation='relu'):
    x = Conv2D(num_filters, 
               kernel_size=kernel_size, 
               strides=strides, 
               padding='same',
               kernel_initializer='he_normal', 
               kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    if (activation):
        x = Activation('relu')(x)
    return x

In [4]:
# 建一个20层的ResNet网络 
def resnet(input_shape):
    inputs = Input(shape=input_shape)# Input层，用来当做占位使用
    
    #第一层
    x = resnet_block(inputs)
    print('layer1,xshape:',x.shape)
    # 第2~7层
    for i in range(6):
        a = resnet_block(inputs = x)
        b = resnet_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out：32*32*16
    # 第8~13层
    for i in range(6):
        if i == 0:
            a = resnet_block(inputs = x,strides=2,num_filters=32)
        else:
            a = resnet_block(inputs = x,num_filters=32)
        b = resnet_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out:16*16*32
    # 第14~19层
    for i in range(6):
        if i ==0 :
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)

        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i == 0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])# 相加操作，要求x、b shape完全一致
        x = Activation('relu')(x)
    # out:8*8*64
    # 第20层   
    x = AveragePooling2D(pool_size=2)(x)
    # out:4*4*64
    y = Flatten()(x)
    # out:1024
    outputs = Dense(10,activation='softmax',
                    kernel_initializer='he_normal')(y)
    
    #初始化模型
    #之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [5]:
# 动态变化学习率
def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5

In [6]:
train_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/train_data/'
test_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/test_data/'

In [7]:
# 声明一个ImageDataGenerator类对象，并给出你需要进行的数据增强选项
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# 调用.flow_from_director()方法，第一个为数据集路径。生成数据集及标签
train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
# 用同样的方法加载测试集
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 50000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [8]:
model = resnet((224, 224, 3))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

layer1,xshape: (None, 224, 224, 16)


In [11]:
checkpoint = ModelCheckpoint(filepath='5deep-resnet.h5', 
                             monitor='val_accuracy', 
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_sch)

lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', 
                               factor=0.2, 
                               patience=5, 
                               mode='max', 
                               min_lr=1e-3)
callbacks = [checkpoint, lr_scheduler, lr_reducer]

history = model.fit_generator(train_generator, 
                              steps_per_epoch = batch_size, 
                              epochs = epochs, 
                              validation_data=test_generator, 
                              verbose=1, 
                              callbacks=callbacks)
score = model.evaluate(test_generator, verbose=0)

Epoch 1/2
1/2 [==============>...............] - ETA: 22s - loss: 116.2640 - accuracy: 0.0938

KeyboardInterrupt: 

In [ ]:
# 打印  loss和acc
print('Test loss', score[0])
print('Test accuracy',score[1])

In [ ]:
# 绘图
import matplotlib.pyplot as plt
%matplotlib inline

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, label='Training accuracy')
plt.plot(epochs, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
    